In [496]:
import os
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [497]:
df = pd.read_csv('ATLAS-Scores only.csv', index_col = 0)
df.drop(['Scientific Name (from Species) 2', 'Average Score'], axis = 1, inplace = True)

In [498]:
# Define relationships between categories and metrics
categories = ['Sustainability', 'Animal Welfare', 'Health', 'US Market Size']
metrics = ['Score: Overall Sustainability', 'Score:  Greenhouse Gas Emissions', 
           'Score: Max Number of Individuals (Log)', 'Score: Edible Coefficient', 
           'Score: Mercury Concentration',  
           'Score: U.S. Retail Sales (Log)', 'Score: U.S. Menu Prevalence', 
           'Score: Import Value (Log)', 'Score: Import Volume (Log)']
metric_dict = {'Sustainability': ['Score: Overall Sustainability', 'Score:  Greenhouse Gas Emissions'],
               'Animal Welfare': ['Score: Edible Coefficient', 'Score: Max Number of Individuals (Log)'],
               'Health': ['Score: Mercury Concentration'],  
               'US Market Size': ['Score: U.S. Retail Sales (Log)', 'Score: U.S. Menu Prevalence', 
               'Score: Import Value (Log)', 'Score: Import Volume (Log)']}

# Widget outputs
output = widgets.Output()
graph_output = widgets.Output()
pie_output = widgets.Output()

# Make sliders
style = {'description_width' : '105px'}
sust_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Sustainability', style = style)
welf_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Animal Welfare', style = style)
health_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Public Health', style = style)
market_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'US Market Size', style = style)

overall_slider = widgets.FloatSlider(value = 0.5, min = 0, max = 1, step = 0.01, description = 'Overall', style = style)
ghg_slider = widgets.FloatSlider(value = 0.5, min = 0, max = 1, step = 0.01, description = 'GHG Emissions', style = style)
individuals_slider = widgets.FloatSlider(value = 0.5, min = 0, max = 1, step = 0.01, description = 'Max # Individuals', style = style)
edible_slider = widgets.FloatSlider(value = 0.5, min = 0, max = 1, step = 0.01, description = 'Edible Portion', style = style)
retail_slider = widgets.FloatSlider(value = 0.33, min = 0, max = 1, step = 0.01, description = 'Retail Sales', style = style)
menu_slider = widgets.FloatSlider(value = 0.33, min = 0, max = 1, step = 0.01, description = 'Menu Prevalence', style = style)
value_slider = widgets.FloatSlider(value = 0.33, min = 0, max = 1, step = 0.01, description = 'Import Value', style = style)
volume_slider = widgets.FloatSlider(value = 0, min = 0, max = 1, step = 0.01, description = 'Import Volume', style = style)

category_sliders = [sust_slider, welf_slider, health_slider, market_slider]
metric_sliders = [overall_slider, ghg_slider, 
                  individuals_slider, edible_slider, 
                  retail_slider, menu_slider, value_slider, volume_slider]
category_slider_dict = {category: slider for category, slider in zip(categories, category_sliders)}
metric_slider_dict = {metric: slider for metric, slider in zip([m for m in metrics if m != 'Score: Mercury Concentration'], metric_sliders)}

def compute_scores():  
    sum_of_cat_weights = sum([slider.value for slider in category_sliders])
    
    category_weights = []
    weight_list = []
    metric_list = []
    
    # Normalize weights within and between categories
    for category in categories:
        category_weight = category_slider_dict[category].value / sum_of_cat_weights
        category_metrics = metric_dict[category]
        if category == 'Health':
            weights = [category_weight]
        else:
            raw_weights = [metric_slider_dict[metric].value for metric in metric_dict[category]]
            sum_of_weights = sum(raw_weights)
            weights = [(raw_weight / sum_of_weights) * category_weight for raw_weight in raw_weights]
        weight_list = weight_list + weights
        metric_list = metric_list + category_metrics
        category_weights.append(category_weight)
    
    # Calculate weighted scores for each metric, fill in missing values with means
    score_breakdown = pd.DataFrame([pd.to_numeric(df[metric]).fillna(pd.to_numeric(df[metric]).mean()) * weight for weight, metric in zip(weight_list, metric_list)]).T

    # Calculate weighted scores only for those with data - this will be used for the bar graph
    score_breakdown_wo_na = pd.DataFrame([pd.to_numeric(df[metric]) * weight for weight, metric in zip(weight_list, metric_list)]).T
    
    # Calculate total scores and sort dataframes by score
    score_breakdown['Score'] = score_breakdown.sum(axis = 1)    
    score_breakdown_wo_na['Score'] = score_breakdown['Score']   
    df['Score'] = score_breakdown['Score']
    
    score_breakdown.sort_values('Score', inplace = True, ascending = False)
    score_breakdown_wo_na.sort_values('Score', inplace = True, ascending = False)
    df.sort_values('Score', inplace = True, ascending = False)    

    # Display ranked list of archetypes. Check whether both import value and volume are weighted and warn user if so.
    with output:
        if value_slider.value != 0 and volume_slider.value != 0:
            display('Warning: Import value and import volume are redundant. You probably should set one or the other to zero.')  
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(df)
    with graph_output:
        if value_slider.value != 0 and volume_slider.value != 0:
            display('Warning: Import value and import volume are redundant. You probably should set one or the other to zero.')  


    # Make color palettes
    sust_colors = sns.cubehelix_palette(start = 1.8, rot = 0, dark = 0.4, light = 0.9, n_colors = 8)
    welf_colors = sns.cubehelix_palette(start = 0.5, rot = 0, dark = 0.4, light = 0.9, n_colors = 8)
    health_colors = sns.cubehelix_palette(start = 3, rot = 0, dark = 0.4, light = 0.9, n_colors = 8)
    market_colors = sns.cubehelix_palette(start = 2.5, rot = 0, dark = 0.4, light = 0.9, n_colors = 8)  
    colors = sust_colors[3:5] + welf_colors[3:5] + health_colors[3:4] + market_colors[3:7]

    # Plot score breakdown as bar graph
    with graph_output:
        bottom = [0] * 20
        ind = np.arange(20)
        score_breakdown = score_breakdown.head(20)
        score_breakdown_wo_na = score_breakdown_wo_na.head(20)
        plt.bar(ind, score_breakdown['Score'], color = 'lightgrey')
        score_breakdown.drop('Score', axis = 1, inplace = True)
        for column, color in zip(score_breakdown.columns, colors):
            plt.bar(ind, score_breakdown_wo_na[column], bottom = bottom, label = column, color = color)
            bottom = bottom + score_breakdown[column]
        plt.xticks(ind, score_breakdown.index[:20], rotation = 90)
        plt.legend(loc = 'upper left', bbox_to_anchor = (1,1))
        plt.show()

    # Display weights as a donut chart
    with pie_output:
        plt.pie(category_weights, colors = [sust_colors[0], welf_colors[0], health_colors[0], market_colors[0]], wedgeprops = {'width' : 0.2}, labels = categories)
        plt.pie(weight_list, colors = colors, radius = 0.75, wedgeprops = {'width' : 0.2})
        plt.show()

# This runs every time you move a slider
def handler(change):
    output.clear_output()
    graph_output.clear_output()
    pie_output.clear_output()
    compute_scores()

for slider in category_sliders + metric_sliders:
    slider.observe(handler, names = 'value')

# Make the layout pretty
sust_layout = widgets.VBox([overall_slider, ghg_slider])
welf_layout = widgets.VBox([individuals_slider, edible_slider])
market_layout = widgets.VBox([retail_slider, menu_slider, value_slider, volume_slider])
metric_layout = widgets.HBox([sust_layout, welf_layout, market_layout])

category_layout = widgets.VBox([sust_slider, welf_slider, health_slider, market_slider])
tab = widgets.Tab([category_layout, metric_layout])
tab.set_title(0, 'Categories')
tab.set_title(1, 'Metrics')

output_tab = widgets.Tab([output, widgets.HBox([graph_output, pie_output])])
output_tab.set_title(0, 'Table')
output_tab.set_title(1, 'Graphs')

# Calculate the initial scores
compute_scores()

In [499]:
display(tab)

In [500]:
display(output_tab)